In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import re
import functools
from geopy.distance import geodesic
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
import numpy as np

from data import get_data
from train import train_loop

In [2]:
train = get_data('data/raw/train1.csv')

/home/ivlazunin/emergency-data-hack/data.py:39: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tmp = tmp.drop(tmp[condition1].index)


In [3]:
clf = train_loop(train)

Размер выборки для обучения:  (121219, 7)
Размер выборки для тестирования:  (30305, 7)
Learning rate set to 0.101061
0:	learn: 0.9443196	total: 66.3ms	remaining: 1m 6s
1:	learn: 0.8291958	total: 79.1ms	remaining: 39.5s
2:	learn: 0.7395402	total: 93.3ms	remaining: 31s
3:	learn: 0.6676451	total: 106ms	remaining: 26.4s
4:	learn: 0.6089990	total: 120ms	remaining: 23.8s
5:	learn: 0.5606801	total: 128ms	remaining: 21.2s
6:	learn: 0.5200001	total: 142ms	remaining: 20.1s
7:	learn: 0.4854084	total: 155ms	remaining: 19.2s
8:	learn: 0.4560945	total: 168ms	remaining: 18.5s
9:	learn: 0.4310553	total: 182ms	remaining: 18s
10:	learn: 0.4097112	total: 195ms	remaining: 17.5s
11:	learn: 0.3912974	total: 208ms	remaining: 17.1s
12:	learn: 0.3754659	total: 221ms	remaining: 16.8s
13:	learn: 0.3620016	total: 231ms	remaining: 16.3s
14:	learn: 0.3504509	total: 241ms	remaining: 15.8s
15:	learn: 0.3402494	total: 254ms	remaining: 15.6s
16:	learn: 0.3316275	total: 267ms	remaining: 15.4s
17:	learn: 0.3240993	total:

In [4]:
test = get_data('data/raw/test1.csv', target=False)

In [5]:
columns = ['volume',
           'occupancy',
           'speed',
           'repair']

In [6]:
test['target'] = clf.predict(test[columns]).astype(int)

In [7]:
test['target'].value_counts()

0    7332582
1       9881
2        691
Name: target, dtype: int64

In [8]:
from time import time
prediction = test[['datetime','road_id','road_km','target']][test['target'] != 0]
prediction.to_csv(f'data/predictions/sol_{int(time())}.csv', index=False)